In [66]:
import pandas as pd
import requests
from io import StringIO
import re
from datetime import date
import time
import yfinance as yf
import numpy as np

# Question 1: [IPO] Withdrawn IPOs by Company Type
What is the total withdrawn IPO value (in $ millions) for the company class with the highest total withdrawal value?

From the withdrawn IPO list ([stockanalysis.com/ipos/withdrawn](https://stockanalysis.com/ipos/withdrawn/)), collect and process the data to find out which company type saw the most withdrawn IPO value.

Steps:
* Use pandas.read_html() with the URL above to load the IPO withdrawal table into a DataFrame. It is a similar process to Code Snippet 1 discussed at the livestream. You should get 99 entries.
* Create a new column called Company Class, categorizing company names based on patterns like:
    - “Acquisition Corp” or “Acquisition Corporation” → Acq.Corp
    - “Inc” or “Incorporated” → Inc
    - “Group” → Group
    - “Ltd” or “Limited” → Limited
    - “Holdings” → Holdings
    - Others → Other

Order: Please follow the listed order of classes and assign the first matched value (e.g., for 'shenni holdings limited', you assign the 'Limited' class).\
Hint: make your function more robust by converting names to lowercase and splitting into words before matching patterns.

* Define a new field Avg. price by parsing the Price Range field (create a function and apply it to the Price Range column).\
  Examples:
    - '$8.00-$10.00' → 9.0\
    - '$5.00' → 5.0\
    - '-' → None
* Convert Shares Offered to numeric, clean missing or invalid values.
* Create a new column: Withdrawn Value = Shares Offered * Avg Price (71 non-null values)
* Group by Company Class and calculate total withdrawn value.
* Answer: Which class had the highest total value of withdrawals?

In [45]:
def get_ipos(url) -> pd.DataFrame:
    """
    Fetch IPO data for the given year from stockanalysis.com.
    """
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/58.0.3029.110 Safari/537.3'
        )
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        # Wrap HTML text in StringIO to avoid deprecation warning
        # "Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object."
        html_io = StringIO(response.text)
        tables = pd.read_html(html_io)

        if not tables:
            raise ValueError(f"No tables found for year {year}.")

        return tables[0]

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except ValueError as ve:
        print(f"Data error: {ve}")
    except Exception as ex:
        print(f"Unexpected error: {ex}")

    return pd.DataFrame()

In [27]:
df = get_ipos("https://stockanalysis.com/ipos/withdrawn/")

In [28]:
def classify_company(name):
    name = name.lower()
    if re.search(r'acquisition (corp|corporation)', name):
        return 'Acq.Corp'
    elif re.search(r'\b(inc|incorporated)\b', name):
        return 'Inc'
    elif 'group' in name:
        return 'Group'
    elif re.search(r'\b(ltd|limited)\b', name):
        return 'Limited'
    elif 'holdings' in name:
        return 'Holdings'
    else:
        return 'Other'

df['Company Class'] = df['Company Name'].apply(classify_company)

In [29]:
def parse_avg_price(price_str):
    if price_str.strip() == '-':
        return None
    # Extract all numbers like 4.00, 6.25, etc.
    prices = re.findall(r'\$?(\d+(?:\.\d+)?)', price_str)
    prices = [float(p) for p in prices]
    if not prices:
        return None
    return sum(prices) / len(prices)

df['Avg Price'] = df['Price Range'].apply(parse_avg_price)

In [30]:
df['Shares Offered'] = pd.to_numeric(df['Shares Offered'].replace('-', None), errors='coerce')

In [31]:
df['Withdrawn Value'] = df['Shares Offered'] * df['Avg Price']

In [37]:
df['Avg Price'].dropna().shape

(73,)

In [43]:
pd.options.display.float_format = '{:,.2f}'.format
df.groupby(['Company Class'])['Withdrawn Value'].sum()

Company Class
Acq.Corp   4,021,000,000.00
Group         33,787,500.00
Holdings      75,000,000.00
Inc        2,257,164,204.85
Limited      549,734,585.25
Other        767,919,999.00
Name: Withdrawn Value, dtype: float64

In [44]:
pd.reset_option('display.float_format')

# Question 2: [IPO] Median Sharpe Ratio for 2024 IPOs (First 5 Months)
What is the median Sharpe ratio (as of 6 June 2025) for companies that went public in the first 5 months of 2024?

The goal is to replicate the large-scale yfinance OHLCV data download and perform basic financial calculations on IPO stocks.

Steps:
* Using the same approach as in Question 1, download the IPOs in 2024 from: https://stockanalysis.com/ipos/2024/ \
Filter to keep only those IPOs before 1 June 2024 (first 5 months of 2024).\
You should have 75 tickers.

* Use Code Snippet 7 to download daily stock data for those tickers (via yfinance).\
Make sure you understand how growth_1d ... growth_365d, and volatility columns are defined.\
Define a new column growth_252d representing growth after 252 trading days (~1 year), in addition to any other growth periods you already track.

* Calculate the Sharpe ratio assuming a risk-free rate of 4.5%:\
stocks_df['Sharpe'] = (stocks_df['growth_252d'] - 0.045) / stocks_df['volatility'] \
⚠️ IMPORTANT Please use the original version of annualized volatility calculation (it was later corrected to another formula):\
stocks_df['volatility'] = stocks_df['Close'].rolling(30).std() * np.sqrt(252) 

* Filter the DataFrame to keep data only for the trading day: ‘2025-06-06’

* Compute descriptive statistics (e.g., .describe()) for these columns:
    - growth_252d
    - Sharpe

    You should observe:
    - growth_252d is defined for 71 out of 75 stocks (some IPOs are too recent or data starts later).
    - Median growth_252d is approximately 0.75 (indicating a 25% decline), while mean is about 1.15, showing a bias towards high-growth companies pushing the average up.

* Answer: What is the median Sharpe ratio for these 71 stocks?\
Note: Positive Sharpe means growth exceeding the risk-free rate of 4.5%.\
[Additional] Do you observe the same top 10 companies when sorting by growth_252d versus sorting by Sharpe?

In [90]:
df = get_ipos("https://stockanalysis.com/ipos/2024/")

In [91]:
df['date'] = pd.to_datetime(df['IPO Date'])
df = df[df['date'] < '2024-06-01']
# invalid rows (companies registered later)
df = df[df['IPO Price'] != '-']

In [93]:
df['Symbol'].nunique()

75

In [94]:
ALL_TICKERS = df['Symbol'].unique()

In [100]:
stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(ALL_TICKERS):
  print(i,ticker)

  # Work with stock prices
  ticker_obj = yf.Ticker(ticker)

  # historyPrices = yf.download(tickers = ticker,
  #                    period = "max",
  #                    interval = "1d")
  historyPrices = ticker_obj.history(
                     period = "max",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  for i in [1,3,7,30,90,365, 252]:
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Close'] / historyPrices['Close'].shift(i)
  historyPrices['growth_future_30d'] = historyPrices['Close'].shift(-30) / historyPrices['Close']

  # Technical indicators
  # SimpleMovingAverage 10 days and 20 days
  historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
  historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
  historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
  historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Close']

  # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  historyPrices['volatility'] = historyPrices['growth_1d'].rolling(30).std() * np.sqrt(252)

  # what we want to predict
  historyPrices['is_positive_growth_30d_future'] = np.where(historyPrices['growth_future_30d'] > 1, 1, 0)

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)

  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

0 BOW
1 HDL
2 RFAI
3 JDZG
4 RAY
5 BTOC
6 ZK
7 GPAT
8 PAL
9 SVCO
10 NNE
11 CCIX
12 VIK
13 ZONE
14 LOAR
15 MRX
16 RBRK
17 NCI
18 MFI
19 YYGH
20 TRSG
21 CDTG
22 CTRI
23 IBTA
24 MTEN
25 TWG
26 ULS
27 PACS
28 MNDR
29 CTNM
30 MAMO
31 ZBAO
32 BOLD
33 MMA
34 UBXG
35 IBAC
36 AUNA
37 BKHA
38 LOBO
39 RDDT
40 ALAB
41 INTJ
42 RYDE
43 LGCL
44 SMXT
45 VHAI
46 DYCQ
47 CHRO
48 UMAC
49 HLXB
50 MGX
51 TBBB
52 TELO
53 KYTX
54 PMNT
55 AHR
56 LEGT
57 ANRO
58 GUTS
59 AS
60 FBLG
61 AVBP
62 BTSG
63 HAO
64 CGON
65 YIBO
66 JL
67 SUGP
68 JVSA
69 KSPI
70 CCTG
71 PSBD
72 SYNX
73 SDHC
74 ROMA


In [101]:
risk_free_rate = 0.045
stocks_df['Sharpe'] = (stocks_df['growth_252d'] - risk_free_rate) / stocks_df['volatility']

In [102]:
filtered_df = stocks_df[stocks_df['Date'] == date(2025,6,6)]

In [103]:
filtered_df[['growth_252d','Sharpe']].describe()

,growth_252d,Sharpe
count,71.000000,71.000000
mean,1.152897,0.288285
std,1.406017,0.519028
min,0.024970,-0.079677
25%,0.293422,0.041215
50%,0.758065,0.083768
75%,1.362736,0.311507
max,8.097413,2.835668


In [115]:
filtered_df[['Ticker','growth_252d','Sharpe']].sort_values('growth_252d', ascending=False).head(10)

,Ticker,growth_252d,Sharpe
20419,JL,8.097413,0.566222
23237,ROMA,6.156406,0.486840
14243,UMAC,4.966533,0.421306
2948,NNE,4.655224,0.080707
4632,RBRK,3.184065,0.019051
16626,AHR,2.483097,0.112576
17969,AS,2.478203,0.028026
4344,MRX,2.300384,0.076540
11479,RDDT,2.225505,0.019543
6964,MTEN,2.210432,0.053200


In [113]:
filtered_df[['Ticker','growth_252d','Sharpe']].sort_values('Sharpe', ascending=False).head(10)

,Ticker,growth_252d,Sharpe
10856,BKHA,1.045881,2.835668
21091,JVSA,1.071076,2.041531
16933,LEGT,1.049407,1.940267
10269,IBAC,1.044611,1.637119
14583,HLXB,1.061404,1.123493
8152,MNDR,1.030769,0.974234
13571,DYCQ,1.059863,0.969321
12105,INTJ,0.661386,0.744512
20419,JL,8.097413,0.566222
5796,TRSG,0.646209,0.518080


# Question 3: [IPO] Fixed Months Holding Strategy
What is the optimal number of months (1 to 12) to hold a newly IPO'd stock in order to maximize average growth?\
(Assume you buy at the close of the first trading day and sell after a fixed number of trading days.)

Goal:
Investigate whether holding an IPO stock for a fixed number of months after its first trading day produces better returns, using future growth columns.

Steps:
* Start from the existing DataFrame from Question 2 (75 tickers from IPOs in the first 5 months of 2024).

* Add 12 new columns:\
future_growth_1m, future_growth_2m, ..., future_growth_12m\
(Assume 1 month = 21 trading days, so growth is calculated over 21, 42, ..., 252 trading days)\
This logic is similar to historyPrices['growth_future_30d'] from Code Snippet 7, but extended to longer timeframes.

* Determine the first trading day (min_date) for each ticker.\
This is the earliest date in the data for each stock.

* Join the data:\
Perform an inner join between the min_date DataFrame and the future growth data on both ticker and date.\
You should end up with 75 records (one per IPO) with all 12 future_growth_... fields populated.

* Compute descriptive statistics for the resulting DataFrame:\
Use .describe() or similar to analyze each of the 12 columns:  
    - future_growth_1m
    - future_growth_2m
    - ...
    - future_growth_12m
      
* Determine the best holding period:\
Find the number of months (1 to 12) where the average (mean) future growth is maximal.\
This optimal month shows an uplift of >1% compared to all others.\
Still, the average return remains less than 1 (i.e., expected return is less than doubling your investment).

In [118]:
base_stocks_df = stocks_df.copy()

In [140]:
stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(ALL_TICKERS):
  print(i,ticker)
  ticker_obj = yf.Ticker(ticker)

  historyPrices = ticker_obj.history(
                     period = "max",
                     interval = "1d")
  historyPrices['Ticker'] = ticker
  historyPrices['Date'] = historyPrices.index.date
  
  for idx, days in enumerate([21 * i for i in range(1,13)]):
      historyPrices[f'future_growth_{idx+1}m'] = historyPrices['Close'].shift(-days) / historyPrices['Close']

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(2)


  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

0 BOW
1 HDL
2 RFAI
3 JDZG
4 RAY
5 BTOC
6 ZK
7 GPAT
8 PAL
9 SVCO
10 NNE
11 CCIX
12 VIK
13 ZONE
14 LOAR
15 MRX
16 RBRK
17 NCI
18 MFI
19 YYGH
20 TRSG
21 CDTG
22 CTRI
23 IBTA
24 MTEN
25 TWG
26 ULS
27 PACS
28 MNDR
29 CTNM
30 MAMO
31 ZBAO
32 BOLD
33 MMA
34 UBXG
35 IBAC
36 AUNA
37 BKHA
38 LOBO
39 RDDT
40 ALAB
41 INTJ
42 RYDE
43 LGCL
44 SMXT
45 VHAI
46 DYCQ
47 CHRO
48 UMAC
49 HLXB
50 MGX
51 TBBB
52 TELO
53 KYTX
54 PMNT
55 AHR
56 LEGT
57 ANRO
58 GUTS
59 AS
60 FBLG
61 AVBP
62 BTSG
63 HAO
64 CGON
65 YIBO
66 JL
67 SUGP
68 JVSA
69 KSPI
70 CCTG
71 PSBD
72 SYNX
73 SDHC
74 ROMA


In [141]:
min_date_df = stocks_df.groupby(['Ticker'])['Date'].min().reset_index()
min_date_stocks_df = stocks_df.merge(min_date_df, on=['Ticker', 'Date'],how='inner')

In [143]:
min_date_stocks_df.shape

(75, 21)

In [146]:
min_date_stocks_df[[f'future_growth_{i}m' for i in range(1,13)]].describe()

,future_growth_1m,future_growth_2m,future_growth_3m,future_growth_4m,future_growth_5m,future_growth_6m,future_growth_7m,future_growth_8m,future_growth_9m,future_growth_10m,future_growth_11m,future_growth_12m
count,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,74.000000,74.000000,71.000000
mean,0.927259,0.940749,0.833988,0.825192,0.803849,0.864268,0.847206,0.833046,0.881847,0.917991,0.883057,0.901665
std,0.346261,0.574267,0.409480,0.401772,0.488226,0.653079,0.712880,0.762355,0.936894,0.911383,0.862902,0.892658
min,0.098947,0.073800,0.060947,0.045368,0.054109,0.061432,0.048274,0.043103,0.033144,0.041357,0.023674,0.038947
25%,0.778984,0.685815,0.511212,0.517233,0.448403,0.384560,0.296870,0.208677,0.226740,0.242424,0.264661,0.229211
50%,0.977000,1.000000,0.927500,0.909091,0.821092,0.802239,0.844875,0.812109,0.822715,0.772592,0.717585,0.659355
75%,1.046509,1.154013,1.069085,1.134300,1.016381,1.093948,1.114468,1.082365,1.049719,1.200678,1.106735,1.136392
max,2.646505,4.874759,2.040000,1.605000,3.213873,3.670520,5.122350,5.171484,6.764933,5.352601,4.445545,4.849711


In [147]:
desc_df = min_date_stocks_df[[f'future_growth_{i}m' for i in range(1,13)]].describe()

In [154]:
desc_df.loc['mean'].idxmax(), desc_df.loc['mean'].max()

('future_growth_2m', np.float64(0.9407486071313915))

# Question 4: [Strategy] Simple RSI-Based Trading Strategy
What is the total profit (in thousands dollar) you would have earned by investing 1000 dollar every time a stock was oversold (RSI < 25)?

Goal:
Apply a simple rule-based trading strategy using the Relative Strength Index (RSI) technical indicator to identify oversold signals and calculate profits.

Steps:
* Run the full notebook from Lecture 2 (33 stocks)

    * Ensure you can generate the merged DataFrame containing:
        - OHLCV data
        - Technical indicators
        - Macro indicators
        - Focus on getting RSI computed using Code Snippets 8 and 9.
    This process is essential and will help during the capstone project.
* ⚠️ IMPORTANT Please use this file to solve the Home Assignment (all next steps)\
  Download precomputed data using this snippet:
```python
import gdown
import pandas as pd

file_id = "1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-"
gdown.download(f"https://drive.google.com/uc?id={file_id}", "data.parquet", quiet=False)
df = pd.read_parquet("data.parquet", engine="pyarrow")
```
* RSI Strategy Setup:

    - RSI is already available in the dataset as a field.
    - The threshold for oversold is defined as RSI < 25.
* Filter the dataset by RSI and date:
```python
rsi_threshold = 25
selected_df = df[
    (df['rsi'] < rsi_threshold) &
    (df['Date'] >= '2000-01-01') &
    (df['Date'] <= '2025-06-01')
]
```
* Calculate Net Profit Over 25 Years:

    - Total number of trades: 1568
    - For each trade, you invest $1000
    - Use the 30-day forward return (growth_future_30d) to compute net earnings:\
    net_income = 1000 * (selected_df['growth_future_30d'] - 1).sum()
* Final Answer:
What is the net income in dollar K (i.e., in thousands of dollars) that could be earned using this RSI-based oversold strategy from 2000–2025?

In [157]:
import gdown
import pandas as pd

file_id = "1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-"
gdown.download(f"https://drive.google.com/uc?id={file_id}", "data.parquet", quiet=False)
df = pd.read_parquet("data.parquet", engine="pyarrow")

Downloading...
From (original): https://drive.google.com/uc?id=1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-
From (redirected): https://drive.google.com/uc?id=1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-&confirm=t&uuid=56963ed3-fc3c-4572-b82a-5e381d9618f9
To: /Users/iuliia/projects/market_zoomcamp/2_data/data.parquet
100%|██████████| 130M/130M [00:15<00:00, 8.53MB/s] 


In [158]:
rsi_threshold = 25
selected_df = df[
    (df['rsi'] < rsi_threshold) &
    (df['Date'] >= '2000-01-01') &
    (df['Date'] <= '2025-06-01')
]

In [160]:
invest_sum = 1000
net_income = invest_sum * (selected_df['growth_future_30d'] - 1).sum()

In [161]:
net_income

np.float64(24295.523125248386)

# Q5. [Exploratory, Optional] Predicting a Positive-Return IPO
Most of the strategies for investing in IPOs deliver negative average and median returns (and even 75% quantiles).

Question:
How would you change the strategy if you want to increase the profitability?

This is an open-ended brainstorming question — propose ideas for identifying IPOs with positive future returns or building a more effective trading strategy.

- Don’t buy on IPO day. Wait for 7–30 trading days to see how the stock settles.
- Buy only those IPOs that hold above their offer price or show strong relative strength.

Only invest in IPOs that:
- Break above their IPO price after an initial base (technical breakout).
- Have rising volume and relative strength index (RSI).
- Combine this with a short-term trading horizon (e.g., 1–3 months).
Rationale: IPOs that regain momentum after stabilization often lead to short-term gains.

Monitor:
- High initial valuations
- Weak early trading
- Lockup dates


Wait 30 days post-IPO, screen for positive momentum, strong fundamentals, and market tailwinds, then allocate with a fixed holding period or trailing stop.